<a href="https://colab.research.google.com/github/bankvis7/cp-axtra-training-lab-3/blob/main/lab4_2_run_finetune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4 Part 2: Run Fine-tuned Models

เนื้อหา:
- การนำโมเดลที่ Fine-tuning ด้วย LoRA ไปใช้งาน

---

## การติดตั้ง Library

In [ ]:
!uv pip install -q unsloth

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

# ขั้นตอนที่ 1: การโหลดโมเดล

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Reubencf/gemma3-goan-finetuned",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

In [ ]:
FastLanguageModel.for_inference(model)

# ขั้นตอนที่ 2: การเตรียม Prompt

In [ ]:
def create_prompt(instruction, input_text=""):
    if input_text:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
"""

# ขั้นตอนที่ 3: การเรียกใช้งานโมเดล

In [ ]:
instruction = "เขียนฟังก์ชันภาษา Python เพื่อคำนวณลำดับฟีโบนัชชี"

prompt = create_prompt(instruction)
inputs = tokenizer(text=prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
  outputs = model.generate(
      **inputs,
      max_new_tokens=128,
      temperature=0.1,
      top_p=0.9,
      do_sample=True,
  )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response.split("### Response:")[1].strip())

----